In [38]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob
from nltk.translate.bleu_score import sentence_bleu
from transformers import DistilBertModel,DistilBertTokenizer
from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer
from tqdm import tqdm

In [2]:
PATH_ROOT = "/home/shaul/workspace/GitHub/SOTA/"
%cd {PATH_ROOT}

/home/shaul/workspace/GitHub/SOTA


In [3]:
df = pd.read_csv("data/combined/combined_dataset.csv").drop(['Unnamed: 0'], axis = 1)

In [4]:
df.head(3)

,annotator,text1,text2,label,dataset,random,duration,total_seconds,pair_id,reduced_label
0,A3BCGN11HDM8QR,"And he sent Eliakim , who was over the househo...","And he sent Eliakim , who was over the house ,...",2,bible_human,0,0 days 00:00:12.000000000,12,pair_0,-1
1,A3SQ00HYQN7FYB,"And he sent Eliakim , who was over the househo...","And he sent Eliakim , who was over the house ,...",3,bible_human,0,0 days 00:00:12.000000000,12,pair_0,0
2,A5WAWW70PYRP,"And he sent Eliakim , who was over the househo...","And he sent Eliakim , who was over the house ,...",4,bible_human,0,0 days 00:07:19.000000000,439,pair_0,1


In [6]:
# # Removing Outliers
# 
# Under the assumption that by removing some of the bad actors in the dataset, the metrics might be more reflective of the human labeling, we want to currently:
# 
# 1. Find out who the bad actors are
# 2. See if there is any overlap from the dif. category of bad actors
# 
# (Removing bad phrases should also be done - but is beyond the scope of this notebook)
# 
# 
# The categories of bad actors are:
# 
# [x] - Slow Annotators - ba_time </br>
# [x] - Greater variance in random datasets than non-random datasets - ba_unvar_annotations </br>
# [x] - Unpopular (disagree with two others often) - ba_unpopular </br>
# [ ] - Inconsistent with sentiment != semantics - ba_semantics </br>

# # Removing Outliers
# 
# Under the assumption that by removing some of the bad actors in the dataset, the metrics might be more reflective of the human labeling,we want to currently:
 
 1. Find out who the bad actors are
 2. See if there is any overlap from the dif. category of bad actors
 
 (Removing bad phrases should also be done - but is beyond the scope of this notebook)
 
 
 The categories of bad actors are:
 
 [x] - Slow Annotators </br>
 [x] - Greater variance in random datasets than non-random datasets </br>
[ ] - Unpopular (disagree with two others often) </br>
 [ ] - Inconsistent with sentiment != semantics </br>

# Removing Outliers
 
Under the assumption that by removing some of the bad actors in the dataset, the metrics might be more reflective of the human labeling,we want to currently:
 
 1. Find out who the bad actors are
 2. See if there is any overlap from the dif. category of bad actors
 
 (Removing bad phrases should also be done - but is beyond the scope of this notebook)
 
 
 The categories of bad actors are:
 
 [x] - Slow Annotators </br>
 [x] - Greater variance in random datasets than non-random datasets </br>
 [ ] - Unpopular (disagree with two others often) </br>
 [ ] - Inconsistent with sentiment != semantics

# Removing Outliers
 
Under the assumption that by removing some of the bad actors in the dataset, the metrics might be more reflective of the human labeling,we want to currently:
 
 1. Find out who the bad actors are
 2. See if there is any overlap from the dif. category of bad actors
 
 (Removing bad phrases should also be done - but is beyond the scope of this notebook)
 
 
 The categories of bad actors are:
 
 [x] - Slow Annotators </br>
 [x] - Greater variance in random datasets than non-random datasets </br>
 [ ] - Unpopular (disagree with two others often) </br>
 [ ] - Inconsistent with sentiment != semantics </br>

# Removing Outliers

Under the assumption that by removing some of the bad actors in the dataset, the metrics might be more reflective of the human labeling, we want to currently:

1. Find out who the bad actors are
2. See if there is any overlap from the dif. category of bad actors

(Removing bad phrases should also be done - but is beyond the scope of this notebook)


The categories of bad actors are:

[x] - Slow Annotators </br>
[x] - Greater variance in random datasets than non-random datasets </br>
[ ] - Unpopular (disagree with two others often) </br>
[ ] - Inconsistent with sentiment != semantics </br>

## Time Outliers

Under the assumption that anyone that takes over the 95 percentile of time.

In [39]:
print(df.total_seconds.describe(percentiles = [.25,.5,.75,.9,.95]))

# ba = bad actor
ba_time = list(df[df.total_seconds > 336].annotator)

count    39660.000000
mean        86.010867
std        427.060791
min          1.000000
25%          7.000000
50%         12.000000
75%         28.000000
90%        169.000000
95%        336.000000
max      14079.000000
Name: total_seconds, dtype: float64


## Unvarianced Annotations
Labelers whos std is too low mean non-random - random difference is too high  

In [40]:
labelers = df[df.random==0].groupby(['annotator'])['label'].agg(['size','mean','std','min','max'])
labelers = labelers[labelers['size']>1]
#df = df[df.annotator.apply(lambda x:x in set(labelers.index))]

labelers_rand = df[df.random==1].groupby(['annotator'])['label'].agg(['size','mean','std','min','max'])
labelers_rand = labelers_rand[labelers_rand['size']>1]
labelers = labelers.join(labelers_rand, rsuffix = '_rand')
labelers['mean_random_gap'] = labelers['mean']-labelers['mean_rand']
labelers['std_ratio'] = labelers['std']/labelers['std_rand']

total_std = df.groupby('annotator')['label'].std()
total_std.name = 'total_std'
labelers = labelers.join(total_std)

In [41]:
ba_unvar_annotations = list(labelers[(labelers.total_std<1) & (labelers.mean_random_gap < 0)].index)

## Unpopular Annotators
Those who over 50% of the time, disagree with the other annotators (in the reduced label)

In [42]:
df_uniquelabels = df.groupby("pair_id")["reduced_label"].nunique()
pairs_twoagree = list(df_uniquelabels[(df.groupby("pair_id")["reduced_label"].nunique() == 2).values].index)
df_twoagree = df[df["pair_id"].isin(pairs_twoagree)]

df_id_reducedlabel = df_twoagree.groupby("pair_id")['reduced_label'].median()
df_twoagree['generally_accepted_label']  = df_id_reducedlabel.values.repeat(3)

In [43]:
df_unpopularopinion = df_twoagree[df_twoagree.reduced_label != df_twoagree.generally_accepted_label].groupby('annotator').size().reset_index()
df_unpopularopinion.columns = ['annotator','unpopular_opinion']

df_allopinions = df[df['annotator'].isin(list(df_unpopularopinion.annotator))].groupby('annotator').size().reset_index()
df_allopinions.columns = ['annotator','all_opinion']

df_opinion_all_unpop = df_allopinions.merge(df_unpopularopinion,on="annotator")

In [44]:
ba_unpopular = list(df_opinion_all_unpop[((df_opinion_all_unpop.unpopular_opinion / df_opinion_all_unpop.all_opinion) > 0.5) & (df_opinion_all_unpop.all_opinion > 4)].annotator)

## Sentiment // Semantic Understanding

In [13]:
# Sentiment analysis pipeline
sentiment_pipe = pipeline("sentiment-analysis")

In [14]:
from tqdm import tqdm

text1_sent,text2_sent =[], []

pbar = tqdm(total = len(df)//100+1, position = 0, leave = True)
for i in range (len(df)//100+1):
    t1_s = sentiment_pipe(df.text1.tolist()[100*i:np.min([100*i+100,len(df)])])
    t2_s = sentiment_pipe(df.text2.tolist()[100*i:np.min([100*i+100,len(df)])])
    text1_sent+=t1_s
    text2_sent+=t2_s
    pbar.update()
pbar.close()
len(text1_sent)

100%|██████████| 397/397 [19:35<00:00,  2.96s/it]


39660

In [15]:
sent = {'POSITIVE':1,'NEGATIVE':-1}

df['sentiment_1'] = np.array([x['score']*sent[x['label']] for x in text1_sent]) 
df['sentiment_2'] = np.array([x['score']*sent[x['label']] for x in text2_sent])
df['dif_sent'] =  np.abs(df['sentiment_1']-df['sentiment_2'])

In [16]:
pairs = []
for index, row in df.iterrows():
    first_sentence_tokens = row['text1'].strip().split()
    second_sentence_tokens = row['text2'].strip().split()
    pairs.append((first_sentence_tokens, second_sentence_tokens))

In [17]:
scores_bleu1 = []
for first_sentence_tokens, second_sentence_tokens in pairs:

    score_bleu1 = sentence_bleu([first_sentence_tokens], second_sentence_tokens, weights=(1, 0, 0, 0))
    scores_bleu1.append(score_bleu1)

print(np.mean(scores_bleu1))
print(np.std(scores_bleu1))


0.305811962880319
0.2615081970765116


In [18]:
df['bleu_score_1'] = scores_bleu1

In [37]:
annot_std_semantic = df[(df['bleu_score_1'] > 0.8) & (df['dif_sent'] > 1.9)].groupby('annotator')['label'].std().dropna()
ba_semantics = list(annot_std_semantic[annot_std_semantic > 1.0].index)

# Combining all the results

In [48]:
import itertools

In [47]:
def jaccard_similarity(list1, list2):
    s1 = set(list1)
    s2 = set(list2)
    return len(s1.intersection(s2)) / len(s1.union(s2))

In [69]:
all_ba = ['ba_semantics','ba_time','ba_unpopular','ba_unvar_annotations']

for a,b in list(itertools.combinations(all_ba,2)):
    print(f"Jaccard Similarity of {a} and {b} is :{jaccard_similarity(eval(a),eval(b))}")

Jaccard Similarity of ba_semantics and ba_time is :0.096
Jaccard Similarity of ba_semantics and ba_unpopular is :0.027777777777777776
Jaccard Similarity of ba_semantics and ba_unvar_annotations is :0.045454545454545456
Jaccard Similarity of ba_time and ba_unpopular is :0.12142857142857143
Jaccard Similarity of ba_time and ba_unvar_annotations is :0.08403361344537816
Jaccard Similarity of ba_unpopular and ba_unvar_annotations is :0.15789473684210525


While the overlap isn't super consistent, it is interesting to note that the two most correlated groups are time and unpopularity and unpopularity with unvaried annotations.

In [65]:
all_ba = list(set(ba_unvar_annotations + ba_unpopular + ba_time + ba_semantics))
print(f"Total number of bad annotators are: {len(all_ba)}")
print(f"Percentage of total annotators are: {len(all_ba)/df.annotator.nunique()}")

Total number of bad annotators are: 160
Percentage of total annotators are: 0.34557235421166305


### Save the annotators so we can filter them out quicker later

In [74]:
for ba in all_ba:
    with open(f'data/other/{ba}.txt', 'w') as f:
        for item in eval(ba):
            f.write("%s\n" % item)

In [75]:
with open('data/other/ba_all.txt','w') as f:
    for item in list(set(ba_unvar_annotations + ba_unpopular + ba_time + ba_semantics)):
        f.write("%s\n" % item)